https://stackabuse.com/python-for-nlp-movie-sentiment-analysis-using-deep-learning-in-keras/

In [ ]:
from keras.preprocessing.text import one_hot
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
# from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
class UkrainianStemmer():
    def __init__(self, word):
        self.word = word
        self.vowel = r'аеиоуюяіїє'  # http://uk.wikipedia.org/wiki/Голосний_звук
        self.perfectiveground = r'(ив|ивши|ившись|ыв|ывши|ывшись((?<=[ая])(в|вши|вшись)))$'
        # http://uk.wikipedia.org/wiki/Рефлексивне_дієслово
        self.reflexive = r'(с[яьи])$'
        # http://uk.wikipedia.org/wiki/Прикметник + http://wapedia.mobi/uk/Прикметник
        self.adjective = r'(ими|ій|ий|а|е|ова|ове|ів|є|їй|єє|еє|я|ім|ем|им|ім|их|іх|ою|йми|іми|у|ю|ого|ому|ої)$'
        # http://uk.wikipedia.org/wiki/Дієприкметник
        self.participle = r'(ий|ого|ому|им|ім|а|ій|у|ою|ій|і|их|йми|их)$'
        # http://uk.wikipedia.org/wiki/Дієслово
        self.verb = r'(сь|ся|ив|ать|ять|у|ю|ав|али|учи|ячи|вши|ши|е|ме|ати|яти|є)$'
        # http://uk.wikipedia.org/wiki/Іменник
        self.noun = r'(а|ев|ов|е|ями|ами|еи|и|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я|і|ові|ї|ею|єю|ою|є|еві|ем|єм|ів|їв|ю)$'
        self.rvre = r'[аеиоуюяіїє]'
        self.derivational = r'[^аеиоуюяіїє][аеиоуюяіїє]+[^аеиоуюяіїє]+[аеиоуюяіїє].*(?<=о)сть?$'
        self.RV = ''

    def ukstemmer_search_preprocess(self, word):
        word = word.lower()
        word = word.replace("'", "")
        word = word.replace("ё", "е")
        word = word.replace("ъ", "ї")
        return word

    def s(self, st, reg, to):
        orig = st
        self.RV = re.sub(reg, to, st)
        return (orig != self.RV)

    def stem_word(self):
        word = self.ukstemmer_search_preprocess(self.word)
        if not re.search('[аеиоуюяіїє]', word):
            stem = word
        else:
            p = re.search(self.rvre, word)
            start = word[0:p.span()[1]]
            self.RV = word[p.span()[1]:]

            # Step 1
            if not self.s(self.RV, self.perfectiveground, ''):

                self.s(self.RV, self.reflexive, '')
                if self.s(self.RV, self.adjective, ''):
                    self.s(self.RV, self.participle, '')
                else:
                    if not self.s(self.RV, self.verb, ''):
                        self.s(self.RV, self.noun, '')
            # Step 2
            self.s(self.RV, 'и$', '')

            # Step 3
            if re.search(self.derivational, self.RV):
                self.s(self.RV, 'ость$', '')

            # Step 4
            if self.s(self.RV, 'ь$', ''):
                self.s(self.RV, 'ейше?$', '')
                self.s(self.RV, 'нн$', u'н')

            stem = start + self.RV
        return stem


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AiBotProject/db.csv', index_col=['id'])
print(df)
df['parsed_text'] = df.parsed_text.str.lower()
df['parsed_text'] = df.parsed_text.apply(lambda x: re.sub(r'[^\w\s]', '', x))
df

In [ ]:
df.iloc[[80, 266]] = None
df.dropna(inplace=True)

In [ ]:
temp = list(map(word_tokenize, df.parsed_text.values))
df['parsed_text'] = temp
# lem = WordNetLemmatizer()
print(temp)
df

[['китайська', 'жіночкакішка', 'продовжує', 'намагатись', 'в', 'пропаганду'], ['1991', 'россия', 'давайте', 'не', 'будем', 'ссориться', 'мы', 'ведь', 'братья', 'украина', 'мы', 'бандеры', 'мы', 'хотим', 'майданы', 'и', 'разруху', 'ущемление', 'русских', 'россия', 'пожалуйста', 'давайте', 'сохраним', 'совок', 'украина', 'иди', 'нахй', '1999', 'украина', 'бомбит', 'югославию', 'россия', 'украина', 'пожалуйста', 'остановись', 'украина', 'иди', 'нахуй', '2004', 'украина', 'проводит', 'майданы', 'и', 'ущемление', 'русских', 'россия', 'давай', 'все', 'будет', 'мирно', 'и', 'мы', 'будем', 'дружить', 'украина', 'иди', 'нахй', '2013', 'украина', 'проводит', 'кровавые', 'незаконные', 'майданы', 'и', 'ущемляет', 'русских', 'россия', 'украина', 'остановись', 'в', 'крыму', 'вообще', 'плохо', 'нам', 'в', 'случае', 'чего', 'придется', 'помогать', 'путя', 'спасибо', 'за', 'крым', 'украина', 'иди', 'нахй', '20142022', 'украина', 'бомбит', 'донбасс', 'и', 'геноцидит', 'русских', 'вступает', 'в', 'нато',

,parsed_text,propaganda,language_code
id,,,
1147,"[китайська, жіночкакішка, продовжує, намагатис...",f,ua
2,"[1991, россия, давайте, не, будем, ссориться, ...",t,rus
4,"[мысли, вслух, каждого, росисянина, сейчас, сл...",t,rus
5,"[комментарий, пескова, по, поводу, диалога, эс...",t,rus
6,"[реакции, мы, решили, включить, ставим, лайки,...",t,rus
...,...,...,...
5182,"[аналiтика, поки, прості, українці, радіють, п...",t,ua
5183,"[чутки, опісля, того, що, сталося, на, євробач...",t,ua
5185,"[інсайд, наше, джерело, доповідає, що, сталася...",t,ua


In [ ]:
def stemming(parsed_text):
  stem_parsed_text = []
  for word in parsed_text:
    ukrainian_stemmer = UkrainianStemmer(word)
    stem_parsed_text.append(ukrainian_stemmer.stem_word())

  return stem_parsed_text
  
print(df.parsed_text)
df['parsed_text'] = df['parsed_text'].apply(lambda x: stemming(x))
print(df.parsed_text)

id
1147    [китайська, жіночкакішка, продовжує, намагатис...
2       [1991, россия, давайте, не, будем, ссориться, ...
4       [мысли, вслух, каждого, росисянина, сейчас, сл...
5       [комментарий, пескова, по, поводу, диалога, эс...
6       [реакции, мы, решили, включить, ставим, лайки,...
                              ...                        
5182    [аналiтика, поки, прості, українці, радіють, п...
5183    [чутки, опісля, того, що, сталося, на, євробач...
5185    [інсайд, наше, джерело, доповідає, що, сталася...
5186    [чутки, згідно, наших, джерел, в, оп, сьогодні...
5187    [інсайд, наш, експерт, ківа, не, просто, так, ...
Name: parsed_text, Length: 2177, dtype: object
id
1147    [китайськ, жіночкакішк, продовж, намаг, в, про...
2       [1991, росс, давайт, не, буд, ссорит, мы, вед,...
4       [мысли, вслух, кажд, росисянин, сейчас, слов, ...
5       [комментар, песк, по, повод, диалог, эстон, и,...
6       [реакц, мы, решил, включит, став, лайк, за, си...
                   

In [ ]:
my_vectorizer = TfidfVectorizer()

In [ ]:
df['parsed_text'] = df['parsed_text'].apply(lambda x: ' '.join(x))
print(df['parsed_text'])
def get_all_words(all_parsed_text):
  for x in all_parsed_text:
    yield x
    # for y in x:
    #   yield y

my_vectorizer.fit(get_all_words(df.parsed_text))

id
1147       китайськ жіночкакішк продовж намаг в пропаганд
2       1991 росс давайт не буд ссорит мы вед брат укр...
4       мысли вслух кажд росисянин сейчас слов админис...
5       комментар песк по повод диалог эстон и зелен п...
6       реакц мы решил включит став лайк за сильн рос ...
                              ...                        
5182    аналiтик пок прост українц радіют просуванн зс...
5183    чутк опісл тог що стал на євробаченн зовс огид...
5185    інсайд наш джерел доповід що стал гаряч сварк ...
5186    чутк згідн наш джерел в оп сьогодн мож бут щас...
5187    інсайд наш експерт ків не прост так говорит за...
Name: parsed_text, Length: 2177, dtype: object


TfidfVectorizer()

In [ ]:
df[df['parsed_text'].str.len() == 0]['parsed_text'] = None
df.dropna(inplace=True)
temp_text = df[df['propaganda']==0][df['language_code']=='ua']
temp_text = temp_text[:300]
df = df.apply(lambda x: None if x.propaganda==0 and x.language_code=='ua' else x)
# df.apply(lambda x)
print(df)
print(temp_text)
# for ls in df.parsed_text:
  # LS.append(my_vectorizer.transform(ls).toarray()).
  # if len(ls) == 0:
print(my_vectorizer.transform(df.iloc[[2, 3]]['parsed_text']).toarray())
print(my_vectorizer.transform(df['parsed_text']).toarray().shape)
x = my_vectorizer.transform(df['parsed_text']).toarray()
y = df['propaganda'].apply(lambda x: 0 if x=='f' else 1).values.astype(int)

x = x[:, np.newaxis, :]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
  #   print("yes")
# print(LS)
# df['parsed_text'] = LS
print(df['parsed_text'])

<ipython-input-9-108f1bfa811f>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_text = df[df['propaganda']==0][df['language_code']=='ua']


AttributeError: ignored

https://www.analyticsvidhya.com/blog/2021/06/lstm-for-text-classification/

In [ ]:
model = Sequential()

model.add(LSTM(128, input_shape=(1, 18472), activation='relu', return_sequences=True))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
# model.add(Dropout(0.2))
# model.add(Dense(32,activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
print(x_train)
print(x_train.shape)
# print(y_train[:3])



In [ ]:
model.fit(x_train, y_train, batch_size=8, epochs=5, validation_split=.2)


In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
my_text = 'донатьте на ЗСУ.'

In [ ]:
my_text = my_text.lower()
my_text = re.sub(r'[^\w\s]', '', my_text)
my_text = word_tokenize(my_text)
my_text = stemming(my_text)
my_text = my_vectorizer.transform([' '.join(my_text)]).toarray()
print(sum(sum(my_text)))
pred = model.predict(my_text[:, np.newaxis, :])
np.round(pred, 2)
